In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

customerprofile = pd.read_csv('CustomerProfile.csv')
Merchant = pd.read_csv('Merchant.csv')
MerchantOffers = pd.read_csv('MerchantOffers.csv')
Transaction = pd.read_csv('Modified_Transaction_Amount_Diverse.csv')
Transaction['MerchantID'] = Transaction['MerchantID'].apply(lambda x : int(x))
Transaction['Amount'] = (Transaction['Amount'] * np.abs(np.random.uniform(0.5, 50, size=len(Transaction))))

FileNotFoundError: [Errno 2] No such file or directory: 'CustomerProfile.csv'

In [ ]:
# from gtts import gTTS

# def arabic_text_to_speech(input_text, output_file="output.mp3"):
#     # Create TTS object
#     tts = gTTS(input_text, lang='ar')

#     # Save the audio file
#     tts.save(output_file)
#     print(f"Audio saved to {output_file}")

# # Example usage
# input_text = "اذيك ياعمي خالد عامل اي "
# arabic_text_to_speech(input_text)


In [ ]:
print(customerprofile.shape)
customerprofile

In [ ]:
print(Merchant.shape)
print(Merchant['MerchantID'].unique())
Merchant.head()

In [ ]:
print(MerchantOffers.shape)
print(Merchant['MerchantID'].unique())
# MerchantOffers['Merchant_ID'] = [3,4,5,0,1,2,6,7,8]
MerchantOffers.head()

In [ ]:
MerchantOffers['Offer_Description']

MerchantOffers['Offer_Description]

In [ ]:
print(Transaction.shape)
print(Transaction['CustomerID'].unique())
Transaction.head()

In [ ]:
# Calculate average transaction amount per customer
customer_avg_amount = Transaction.groupby('CustomerID')['Amount'].agg(['mean', 'median', 'std']).reset_index()
customer_profile = customerprofile.merge(customer_avg_amount,  how='left',on = 'CustomerID')
customer_profile

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a boxplot for the distribution of transaction amounts for each customer
plt.figure(figsize=(15, 8))
sns.boxplot(data=Transaction, x='CustomerID', y='Amount')
plt.title("Distribution of Transaction Amounts by Customer")
plt.xlabel("Customer ID")
plt.ylabel("Transaction Amount")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Filter for a specific customer (e.g., CustomerID = 1)
customer_id = 1
customer_transactions = Transaction[Transaction['CustomerID'] == customer_id]

# Plot the histogram
plt.figure(figsize=(10, 6))
plt.hist(customer_transactions['Amount'], bins=20, alpha=0.7, color='blue', edgecolor='black')
plt.title(f"Transaction Amount Distribution for Customer {customer_id}")
plt.xlabel("Transaction Amount")
plt.ylabel("Frequency")
plt.show()


In [ ]:
plt.figure(figsize=(15, 8))
sns.kdeplot(data=Transaction, x='Amount', hue='CustomerID', fill=True, palette='viridis')
plt.title("KDE of Transaction Amounts for All Customers")
plt.xlabel("Transaction Amount")
plt.ylabel("Density")
plt.show()


In [ ]:
g = sns.FacetGrid(Transaction, col="CustomerID", col_wrap=4, height=4, sharex=False, sharey=False)
g.map(sns.histplot, "Amount", bins=15, kde=True, color="blue")
g.set_titles("Customer ID: {col_name}")
g.set_axis_labels("Transaction Amount", "Frequency")
plt.subplots_adjust(top=0.9)
g.fig.suptitle("Transaction Amount Distribution per Customer")
plt.show()


In [ ]:
plt.figure(figsize=(15, 8))
sns.violinplot(data=Transaction, x='CustomerID', y='Amount', scale='width', inner='quartile', palette='muted')
plt.title("Distribution of Transaction Amounts by Customer (Violin Plot)")
plt.xlabel("Customer ID")
plt.ylabel("Transaction Amount")
plt.xticks(rotation=45)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Group the data by CustomerID and plot a histogram for each
customer_ids = Transaction['CustomerID'].unique()

# Plot distribution for the first few customers
for customer_id in customer_ids[:5]:  # Limit to 5 customers for readability
    customer_data = Transaction[Transaction['CustomerID'] == customer_id]['Amount']
    plt.figure(figsize=(8, 4))
    sns.histplot(customer_data, kde=True, bins=10, color='blue')
    plt.title(f"Transaction Amount Distribution for Customer {customer_id}")
    plt.xlabel("Transaction Amount")
    plt.ylabel("Frequency")
    plt.show()


In [ ]:
# Box plot for selected customers
selected_customers = Transaction['CustomerID'].unique()[:10]  # Limit to 10 customers for visualization
filtered_data = Transaction[Transaction['CustomerID'].isin(selected_customers)]

plt.figure(figsize=(12, 6))
sns.boxplot(data=filtered_data, x='CustomerID', y='Amount')
plt.title("Transaction Amount Distribution Across Selected Customers")
plt.xlabel("CustomerID")
plt.ylabel("Transaction Amount")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# KDE plot for individual customers
for customer_id in customer_ids[:5]:  # Limit to 5 customers
    customer_data = Transaction[Transaction['CustomerID'] == customer_id]['Amount']
    plt.figure(figsize=(8, 4))
    sns.kdeplot(customer_data, shade=True)
    plt.title(f"Transaction Amount KDE for Customer {customer_id}")
    plt.xlabel("Transaction Amount")
    plt.ylabel("Density")
    plt.show()


In [15]:
Transaction['TransactionDateTime'] = pd.to_datetime(Transaction['TransactionDateTime'])
MerchantOffers['Offer_Start_Date'] = pd.to_datetime(MerchantOffers['Offer_Start_Date'])
MerchantOffers['Offer_End_Date'] = pd.to_datetime(MerchantOffers['Offer_End_Date'])

In [16]:
customer_transaction = pd.merge(customerprofile, Transaction, on='CustomerID', how='inner')

In [17]:
customer_transaction_merchant = pd.merge(customer_transaction, Merchant, on='MerchantID', how='inner')
full_data = pd.merge(customer_transaction_merchant, MerchantOffers, on='MerchantID', how='left')

In [18]:
full_data['TotalSpent'] = full_data.groupby('CustomerID')['Amount'].transform('sum')
full_data['AvgTransactionAmount'] = full_data.groupby('CustomerID')['Amount'].transform('mean')
full_data['TransactionCount'] = full_data.groupby('CustomerID')['TransactionID'].transform('count')
full_data['Hour'] = full_data['TransactionDateTime'].dt.hour
full_data['DayOfWeek'] = full_data['TransactionDateTime'].dt.dayofweek

In [ ]:
full_data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(full_data['Age'], kde=True)
plt.title('Distribution of Customer Ages')
plt.show()

sns.scatterplot(x='Age', y='TotalSpent', data=full_data)
plt.title('Age vs Total Spent')
plt.show()

In [21]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

features = full_data[['Age', 'IncomeEstimation', 'TotalSpent', 'TransactionCount', 'AvgTransactionAmount']]
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

kmeans = KMeans(n_clusters=5, random_state=42)
full_data['Cluster'] = kmeans.fit_predict(features_scaled)

In [ ]:
full_data

In [ ]:
full_data['Offer_ID'].fillna('No Offer', inplace=True)
offer_popularity = full_data.groupby(['Cluster', 'Offer_ID'])['CustomerID'].count().reset_index()
offer_popularity.rename(columns={'CustomerID': 'Count'}, inplace=True)

In [ ]:
offer_popularity

In [ ]:
def recommend_offer(customer_id, data):
    customer_cluster = data[data['CustomerID'] == customer_id]['Cluster'].values[0]
    top_offers = offer_popularity[offer_popularity['Cluster'] == customer_cluster].sort_values(by='Count', ascending=False)['Offer_ID']
    return top_offers.head(3).tolist()

recommended_offers = recommend_offer(12345, full_data)
print(f"Recommended Offers: {recommended_offers}")

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(full_data, lat='LocationLatitude', lon='LocationLongitude', hover_name='Merchant_Name',
                        color='Cluster', size='Amount',
                        zoom=3, height=600)
fig.update_layout(mapbox_style='open-street-map')
fig.show()

In [ ]:
pip install plotly

In [ ]:
customer_profile['Amount'].plot(kind='hist')

In [ ]:
# Calculate total number of transactions per customer
customer_total_transactions = Transaction.groupby('CustomerID')['TransactionID'].count().reindex()
customer_total_transactions


In [ ]:
customer_profile = customer_profile.merge(customer_total_transactions,on='CustomerID',how='left')
customer_profile

In [ ]:
# Calculate frequency of transactions in different MCC categories
mcc_frequency = Transaction.groupby(['CustomerID','MCC'])['TransactionID'].count().reset_index()
mcc_frequency

In [11]:
mcc_pivot = mcc_frequency.pivot(index='CustomerID', columns='MCC', values='TransactionID').fillna(0)

In [ ]:
mcc_pivot

In [13]:
customer_profile = customer_profile.merge(mcc_pivot, on='CustomerID', how='left')

In [ ]:
customer_profile

In [1]:
pip install pandas faker


   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/1.9 MB 2.8 MB/s eta 0:00:01
   --------------------------------- ------ 1.6/1.9 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from faker import Faker

# Initialize Faker for generating realistic data
fake = Faker()

# Define the number of transactions
num_transactions = 500

# Define the list of Egyptian cities
egyptian_cities = ['Cairo', 'Alexandria', 'Giza', 'Luxor', 'Aswan', 'Port Said', 'Suez', 'Mansoura', 'Tanta', 'Fayoum']

# Generate synthetic data
data = {
    'TransactionID': range(num_transactions),
    'CustomerID': np.random.randint(0, 5, num_transactions),  # Customer ID from 0 to 4
    'Amount': np.random.uniform(10, 5000, num_transactions).round(2),  # Variety of amounts
    'LocationCity': np.random.choice(egyptian_cities, num_transactions),  # Random Egyptian cities
    'TransactionDateTime': [fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d %H:%M:%S') for _ in range(num_transactions)],
    'AccountBalanceAfter': np.random.uniform(100, 10000, num_transactions).round(2),  # Random account balances
    'MerchantID': np.random.randint(1, 100, num_transactions),  # Random merchant IDs
    'MCC': np.random.choice([5812, 5411, 5651, 5732, 5912, 5941, 5999], num_transactions),  # Random MCC codes
    'AccountType': np.random.choice(['CREDIT_CARD', 'DEBIT'], num_transactions)  # Only CREDIT_CARD or DEBIT
}

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('Transaction_Egypt.csv', index=False)

print("Data generated and saved to 'Transaction_Egypt.csv'") 

Data generated and saved to 'Transaction_Egypt.csv'
